Shada Habiballah - 212260814


In [1]:
%reload_ext tensorboard
import datetime, os
import tensorflow as tf
import tensorflow_datasets as tfds
class_num = 10
image_shape = (28, 28, 1)

Load Dataset

In [18]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist', split=['train', 'test'], shuffle_files=True,
    as_supervised=True, with_info=True,)


Preprocess the training dataset

In [19]:
def normalize_img(image, label):
    """Normalizes images: uint8 -> float32."""
    return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(64)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
        normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

Network

In [46]:
#212260814
layers = [

      tf.keras.layers.Flatten(input_shape=image_shape),

      tf.keras.layers.Dense(14),
      tf.keras.layers.Activation('elu'),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.BatchNormalization(),

      tf.keras.layers.Dense(8),
      tf.keras.layers.Activation('elu'),
      tf.keras.layers.BatchNormalization(),


      tf.keras.layers.Dense(26),
      tf.keras.layers.Activation('elu'),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.BatchNormalization(),


      tf.keras.layers.Dense(12),
      tf.keras.layers.Activation('elu'),
      tf.keras.layers.BatchNormalization(),


      tf.keras.layers.Dense(class_num, activation='softmax')
]

In [47]:
model = tf.keras.models.Sequential(layers)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.summary()


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_14 (Flatten)                 │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_66 (Dense)                     │ (None, 14)                  │          10,990 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_53 (Activation)           │ (None, 14)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_26 (Dropout)                 │ (None, 14)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_46               │ (None, 14)                  │              56 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_67 (Dense)                     │ (None, 8)                   │             120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_54 (Activation)           │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_47               │ (None, 8)                   │              32 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_68 (Dense)                     │ (None, 26)                  │             234 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_55 (Activation)           │ (None, 26)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_27 (Dropout)                 │ (None, 26)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_48               │ (None, 26)                  │             104 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_69 (Dense)                     │ (None, 12)                  │             324 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_56 (Activation)           │ (None, 12)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_49               │ (None, 12)                  │              48 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_70 (Dense)                     │ (None, 10)                  │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,038 (47.02 KB)

 Trainable params: 11,918 (46.55 KB)

 Non-trainable params: 120 (480.00 B)

Train the model

In [48]:
model.fit(ds_train,
          epochs=15,
          validation_data=ds_test)

Epoch 1/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 1.0222 - sparse_categorical_accuracy: 0.6893 - val_loss: 0.3297 - val_sparse_categorical_accuracy: 0.9039
Epoch 2/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.4760 - sparse_categorical_accuracy: 0.8547 - val_loss: 0.2770 - val_sparse_categorical_accuracy: 0.9172
Epoch 3/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.4102 - sparse_categorical_accuracy: 0.8766 - val_loss: 0.2502 - val_sparse_categorical_accuracy: 0.9257
Epoch 4/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.3889 - sparse_categorical_accuracy: 0.8821 - val_loss: 0.2330 - val_sparse_categorical_accuracy: 0.9285
Epoch 5/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.3666 - sparse_categorical_accuracy: 0.8882 - val_loss: 0.2183 - val_sparse_categorical_accuracy: 0.9327
Epoch 6/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.3523 - sparse_categorical_accuracy: 0.8922 - val_loss: 0.2174 - val_sparse_categorical_accuracy: 0.9337
Epoc